In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Step 1: Create a synthetic full image
full_image = np.zeros((400, 400, 3), dtype=np.uint8)
cv2.rectangle(full_image, (50, 50), (350, 350), (255, 255, 255), -1)
cv2.circle(full_image, (200, 200), 80, (0, 0, 255), -1)
cv2.putText(full_image, 'AI', (150, 215), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 0), 3)

# Step 2: Create a query image by cropping a part of the full image
query_image = full_image[120:280, 120:280]

# Step 3: Convert images to grayscale
img1 = cv2.cvtColor(query_image, cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(full_image, cv2.COLOR_BGR2GRAY)

# Step 4: Use SIFT to detect and compute keypoints and descriptors
sift = cv2.SIFT_create()
kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)

# Step 5: Use FLANN-based matcher
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params, search_params)

matches = flann.knnMatch(des1, des2, k=2)

# Step 6: Apply Lowe's ratio test
good = []
for m, n in matches:
    if m.distance < 0.7 * n.distance:
        good.append(m)

# Step 7: Match found
MIN_MATCH_COUNT = 10
result = full_image.copy()

if len(good) > MIN_MATCH_COUNT:
    src_pts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    matchesMask = mask.ravel().tolist()
    h, w = img1.shape
    pts = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1, 1, 2)
    dst = cv2.perspectiveTransform(pts, M)
    result = cv2.polylines(result, [np.int32(dst)], True, (0, 255, 0), 3, cv2.LINE_AA)
else:
    print("Not enough matches are found - {}/{}".format(len(good), MIN_MATCH_COUNT))

# Step 8: Display the result
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.imshow(cv2.cvtColor(query_image, cv2.COLOR_BGR2RGB))
plt.title("Query Image")

plt.subplot(1, 3, 2)
plt.imshow(cv2.cvtColor(full_image, cv2.COLOR_BGR2RGB))
plt.title("Full Image")

plt.subplot(1, 3, 3)
plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
plt.title("Result")

plt.tight_layout()
plt.show()

